In [ ]:
# Run this cell before anything else to install required packages
!pip install transformers datasets shap seaborn

# Problem 2. Linear Sentiment

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
import shap

In [ ]:
# Load IMDB dataset and split into train/test set
corpus,y = shap.datasets.imdb()
corpus_train, corpus_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=7)
vectorizer = TfidfVectorizer(min_df=10)
X_train = vectorizer.fit_transform(corpus_train).toarray()
X_test = vectorizer.transform(corpus_test).toarray()

In [ ]:
## TODO: Problem 2.(a) Print the first 10 reviews in the test set, and examine
# the corresponding labels

In [ ]:
## TODO: examine the vectorizer.vocabulary_ dictionary

In [ ]:
## TODO: Select any 10 words which appear in the first sentence and in the 
# dictionary, and which you’ll be interested in examining SHAP values for. 
# Then, select their indices using the dictionary, and save them into an idxs
# variable.

In [ ]:
## TODO: Problem 2.(b) Fit a logistic regression model 

In [ ]:
## TODO: Problem 2.(c) Manual Computation of Linear SHAP values

In [ ]:
## TODO: Problem 2.(d) Use shap.LinearExplainer to compute SHAP values.
# Compare results to (c)

# Problem 3. SHAP with ConvNets

In [ ]:
import torch, torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F

import numpy as np
import shap

In [ ]:
# Code to train a ConvNet on MNIST image classification
batch_size = 128
num_epochs = 2
device = torch.device('cpu')
torch.manual_seed(0)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size=5),
            nn.Dropout(),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output.log(), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output.log(), target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
                       transforms.ToTensor()
                   ])),
    batch_size=batch_size, shuffle=True)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312237
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.256531
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.771387
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.962741
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.771190

Test set: Average loss: 0.0051, Accuracy: 8856/10000 (89%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.638823
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.636632
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.628619
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.487452
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.577050

Test set: Average loss: 0.0027, Accuracy: 9303/10000 (93%)



In [ ]:
# Get data for SHAP
batch = next(iter(test_loader))
images, labels = batch

background = images[:100]
test_images = images[110:116]
test_labels = labels[110:116]
pred_labels = model(test_images).max(1, keepdim=True)[1].reshape(-1)

In [ ]:
# TODO: Problem 3. (a) Fit DeepExplainer 

In [ ]:
# TODO: Problem 3. (b) Visualize the explanations

In [ ]:
# TODO: Problem 3. (c) Compare true and predicted labels to the visualization